In [ ]:
import numpy as np
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from utils.orient import OrientUtils
from evaluate.evaluator import Evaluator
from utils.visualize import SearchPlotter
from utils.image import ImageUtils

import mealpy
import cv2 as cv

plotter = SearchPlotter(update_freq=100)

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/mug/world.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/mug/world_sim.xml", cam_config)

loss_func = loss_funcs.IOU()

alg = UniformSampling(sim_viewer, loss_func=loss_func)
alg_config = UniformSampling.Config(time_limit=200, min_samples=350)

eval_func=eval_funcs.XorDiff(0.1)
evaluator = Evaluator(world_viewer, eval_func=eval_func)

init_location = (0, 1.3, 0.3)
random_orientations = OrientUtils.generate_random(80)
eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

for pos in eval_positions:
    orig_img, new_pos = world_viewer.get_view_cropped(pos)
    orient, loss = alg.solve(orig_img, new_pos.location, alg_config)
    pred_img, _ = world_viewer.get_view_cropped(ObjectPosition(orient, new_pos.location))

    cv.imshow("Original", orig_img)
    cv.waitKey(0)
    cv.imshow("Predicted", pred_img)
    cv.waitKey(0)

    orig_depth, _ = world_viewer.get_view_cropped(new_pos, depth=True, allow_simulation=False)
    pred_depth, _ = world_viewer.get_view_cropped(ObjectPosition(orient, new_pos.location), depth=True, allow_simulation=False)

    eval_loss = eval_func(orig_depth, pred_depth)
    print(eval_loss)

for alg, config in [(alg, alg_config)]:
    eval_losses = evaluator.evaluate(alg, config, eval_positions)
    print(f"{type(alg).__name__}: {eval_losses}")


In [ ]:
import numpy as np
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from utils.orient import OrientUtils
from evaluate.evaluator import Evaluator
# from utils.visualize import SearchPlotter
from utils.image import ImageUtils

import mealpy
import cv2 as cv

# plotter = SearchPlotter(update_freq=100, alpha=[0.985])

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.3), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/hammer/world.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/hammer/world_sim.xml", cam_config)

loss_func = loss_funcs.IOU()
alg_config = MealAlgorithm.Config(time_limit=15, silent=True)

eval_func=eval_funcs.XorDiff(0.1)

evaluator = Evaluator(world_viewer, eval_func=eval_func)
# evaluator.register_callback(lambda x: plotter.reset())

init_location = (0, 1.3, 0.3)
random_orientations = OrientUtils.generate_uniform(3)
eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

algorithms: list[MealAlgorithm] = [
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.OriginalSA(temp_init=10 * 2 * np.pi, step_size=0.1)),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.SwarmSA(temp_init=10 * 2 * np.pi, step_size=np.pi/100)),
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.OriginalPSO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.AIW_PSO()),
]
# meal_alg = mealpy.human_based.BRO.DevBRO()
# meal_alg = mealpy.math_based.CEM.OriginalCEM()
# meal_alg = mealpy.music_based.HS.DevHS(pop_size=10)
# meal_alg = mealpy.swarm_based.PSO.OriginalPSO()
# meal_alg = mealpy.physics_based.SA.OriginalSA(pop_size=10, temp_init=10 * 2 * np.pi, step_size=0.2)
# meal_alg = mealpy.human_based.ICA.OriginalICA(empire_count=7, revolution_rate=0.4) # best!!
# meal_alg = mealpy.bio_based.BBO.DevBBO() # potentially good!
meal_alg = mealpy.bio_based.TSA.OriginalTSA() # potentially good!

algorithms: list[MealAlgorithm] = [
    MealAlgorithm(sim_viewer, loss_func, meal_alg),
]

# evaluator.enable_logging("Ealuations/UniformDet/Mug/IOU/")
for alg in algorithms:
    # alg.register_callback(plotter.add_data)
    eval_losses = evaluator.evaluate(alg, alg_config, eval_positions)
    print(f"{alg.get_name()}: {eval_losses}")

world_viewer.close()
sim_viewer.close()
# plotter.close()

In [ ]:
import loss_funcs
from algs import *

import mealpy
from multiprocessing import Process
import evaluation_process as ev

loss_func = loss_funcs.IOU()
alg_config = MealAlgorithm.Config(time_limit=5, silent=True)
root_path = "Ealuations/OverNight/UniformDet/Hammer/IOU/"

if __name__ == '__main__':
    for idx, (alg_name, meal_alg) in enumerate(mealpy.get_all_optimizers().items()):
        print(f"===================== Epoch {idx} =====================")
        print(f"Starting Evaluation of: {alg_name}")
        p = Process(target=ev.evaluate, args=(meal_alg,alg_config,loss_func,root_path))
        p.start()
        p.join()


In [ ]:
import numpy as np
from utils.orient import OrientUtils
len(OrientUtils.generate_uniform(9))

# Analisys

## Plotting Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib qt

In [ ]:
from evaluate.mealpy_log import MealLog
from mealpy.utils.history import History

hs_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/Ealuations/UniformDet/Mug/IOU/DevHS_20240318-191420.pickle"
pso_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/Ealuations/UniformDet/Mug/IOU/OriginalPSO_20240318-194706.pickle"
bro_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/Ealuations/UniformDet/Mug/IOU/DevBRO_20240318-201942.pickle"

logHS = MealLog.load(hs_path)
logPSO = MealLog.load(pso_path)
logBRO = MealLog.load(bro_path)




In [ ]:

hs_loss_list = logHS.eval_loss_list
pso_loss_list = logPSO.eval_loss_list
bro_loss_list = logBRO.eval_loss_list

n_epochs = len(hs_loss_list)
# sns.displot(x=range(n_epochs), y=hs_loss_list)
# sns.displot(x=range(n_epochs), y=pso_loss_list)

data = {"alg": [*(['HS']*n_epochs), *(['PSO']*n_epochs), *(['BRO']*n_epochs)],
        "loss":[*hs_loss_list, *pso_loss_list, *bro_loss_list], 
        "range":[*range(n_epochs),*range(n_epochs),*range(n_epochs)]}
data = pd.DataFrame(data)






In [ ]:
sns.catplot(data=data, x='range', y='loss', hue='alg', kind='bar')

In [ ]:
sns.catplot(data=data, x='alg', y='loss', hue='alg', kind='violin')